In [ ]:
%run data.ipynb

In [ ]:
import pandas as pd
import glob

df_wise_reconciliation = df_wise.copy()
df_wise_reconciliation = df_wise_reconciliation[df_wise_reconciliation['wise.status'] != 'funds_refunded']

print(df_wise_reconciliation.info())
df_wise_reconciliation.head()

In [ ]:
df_platform_wise_related = df_platform[(df_platform['kind'] == 'EXPENSE') & (df_platform['payoutMethod.type'] == 'BANK_ACCOUNT') & (df_platform['type'] == 'DEBIT') & (df_platform['isRefunded'] == False)]
print(df_platform_wise_related.info())
df_platform_wise_related.head()

In [ ]:
# merged_df = pd.merge(df_wise_reconciliation, df_platform_wise[['merchantId', 'id']].dropna(subset=['merchantId']).astype({'merchantId': 'int64'}), left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))
df_platform_wise = pd.merge(df_platform_wise_related.dropna(subset=['merchantId']).astype({'merchantId': 'int'}), df_wise_reconciliation, left_on='merchantId', right_on='wise.id', how='left').dropna(subset=['wise.id'])
print('Successfully merged {} rows out of {}'.format(len(df_platform_wise), len(df_platform_wise_related)))

df_left_platform = df_platform_wise_related[~df_platform_wise_related['id'].isin(df_platform_wise['id'])]
df_left_wise = df_wise_reconciliation[~df_wise_reconciliation['wise.id'].isin(df_platform_wise['wise.id'])]

from utils import matchByDateAndValue
df_second_pass = matchByDateAndValue(df_left_platform, ['createdAt', 'amountInHostCurrency.value'], df_left_wise, ['wise.created', 'wise.sourceValue'], timeDelta='2 days', valueDelta=0.05, absolute=True).dropna(subset=['df2key'])
df_platform_wise = pd.concat([df_platform_wise, df_second_pass.loc[:, df_second_pass.columns!='df2key']], ignore_index=True)
print('Successfully merged {} rows out of {} from Value and Date'.format(len(df_second_pass), len(df_platform_wise_related)))

print(f"Platform transactions that have a corresponding Wise transaction: {len(df_platform_wise)}")
print(f"Platform transactions without a corresponding Wise transaction: {len(df_left_platform)}")
print(f"Percentage of Platform transactions that have a corresponding Wise transaction: {round(len(df_platform_wise)/len(df_platform_wise_related)*100, 2)}%")
print(f"Wise transactions without a corresponding platform transaction: {len(df_left_wise)}")
print(f"Percentage of Wise transactions that have a corresponding Platform transaction: {round(len(df_platform_wise)/len(df_wise_reconciliation)*100, 2)}%")


In [ ]:
df_second_pass.head()

In [ ]:
df_left_platform

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
WISE_TRACE_ACCOUNT = 'XXXXXX7206'
df_banks2 = df_banks[df_banks['Detail'].str.contains(WISE_TRACE_ACCOUNT)]
df_banks2.sample()


In [ ]:
def matchByDateAndValue(df1, df1cols, df2, df2cols, timeDelta = '10 days', valueDelta = 0.1):
    a = df1[df1cols]
    b = df2[df2cols]
    matched = []
    def naive_transaction_matcher(x):
        possible = b.drop(index=matched)
        possibleByDate = (possible[df2cols[0]] - x[df1cols[0]]).abs().sort_values(ascending=True)
        possibleByDate = possibleByDate[possibleByDate < pd.Timedelta(timeDelta)]
        if len(possibleByDate) == 0:
            return None
        possible = possible.loc[possibleByDate.index]
        possibleByAmount = (possible[df2cols[1]] - x[df1cols[1]]).abs().sort_values(ascending=True)
        possibleByAmount = possibleByAmount[possibleByAmount < possible.loc[possibleByAmount.index][df2cols[1]] * valueDelta]
        if len(possibleByAmount) == 0:
            return None
        id = int(possibleByAmount.index[0])
        matched.append(id)
        return id
    indexes = pd.DataFrame(a.apply(naive_transaction_matcher, axis=1).dropna().astype('int'), columns=["df2key"])
    return df1.join(indexes).join(df2, on="df2key")

df_wise_bank = matchByDateAndValue(df_left_wise, ['wise.created', 'wise.sourceValue'], df_banks2, ['Post Date', 'Debit'] , '2 days', 0.05).dropna(subset=['df2key'])
df_left_wise = df_left_wise[~df_left_wise['wise.id'].isin(df_wise_bank['wise.id'])]
print(f"Wise transactions with a corresponding Bank transaction: {len(df_wise_bank)}")
print(f"Wise transactions without either a corresponding Platform transaction or a Bank transaction: {len(df_left_wise)} out of {len(df_wise_reconciliation)}")
print(f"Percentage of Wise transactions that have a corresponding transaction either on the Platform or on the Bank: {round((len(df_wise_bank)+len(df_platform_wise))/len(df_wise_reconciliation)*100, 2)}%")
df_left_wise.head()